In [7]:
import cv2

# Provide the full paths to the .cfg and .weights files
config_path = "C:/Users/pooja/Downloads/yolov3.cfg"
weights_path = "C:/Users/pooja/Downloads/yolov3.weights"

# Load YOLO model
net = cv2.dnn.readNet(weights_path, config_path)

# Continue with your object detection code...

In [8]:
import os
print(os.getcwd())

C:\Users\pooja


In [9]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
layer_names = net.getLayerNames()
output_layers = net.getUnconnectedOutLayers()
print("Layer Names:", layer_names)
print("Output Layers:", output_layers)

Layer Names: ('conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2', 'conv_2', 'bn_2', 'leaky_3', 'conv_3', 'bn_3', 'leaky_4', 'shortcut_4', 'conv_5', 'bn_5', 'leaky_6', 'conv_6', 'bn_6', 'leaky_7', 'conv_7', 'bn_7', 'leaky_8', 'shortcut_8', 'conv_9', 'bn_9', 'leaky_10', 'conv_10', 'bn_10', 'leaky_11', 'shortcut_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'shortcut_15', 'conv_16', 'bn_16', 'leaky_17', 'conv_17', 'bn_17', 'leaky_18', 'shortcut_18', 'conv_19', 'bn_19', 'leaky_20', 'conv_20', 'bn_20', 'leaky_21', 'shortcut_21', 'conv_22', 'bn_22', 'leaky_23', 'conv_23', 'bn_23', 'leaky_24', 'shortcut_24', 'conv_25', 'bn_25', 'leaky_26', 'conv_26', 'bn_26', 'leaky_27', 'shortcut_27', 'conv_28', 'bn_28', 'leaky_29', 'conv_29', 'bn_29', 'leaky_30', 'shortcut_30', 'conv_31', 'bn_31', 'leaky_32', 'conv_32', 'bn_32', 'leaky_33', 'shortcut_33', 'conv_34', 'bn_34', 'leaky_35', 'conv_35', 'bn_35', 'leaky_36', 'shortcut_36', 'conv_37', 'bn

In [13]:
import cv2

# Step 1: Load YOLO Model
config_path = "C:/Users/pooja/Downloads/yolov3.cfg"
weights_path = "C:/Users/pooja/Downloads/yolov3.weights"
net = cv2.dnn.readNet(weights_path, config_path)

In [14]:
# Step 2: Load Image
image_path = "C:/Users/pooja/Downloads/image.jpg"  # Replace with your image path
img = cv2.imread(image_path)
height, width, _ = img.shape  # Get image dimensions

In [15]:
# Step 3: Prepare Image for YOLO
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)

In [16]:
# Step 4: Get Output Layer Names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]  # Adjust the indexing

In [17]:
# Step 5: Run Forward Pass to Get Predictions
outputs = net.forward(output_layers)

In [18]:
# Step 6: Initialize Lists for Bounding Boxes and Confidences
boxes = []
confidences = []

In [24]:
# Step 7: Process the Outputs
for output in outputs:
    for detection in output:
        scores = detection[5:]  # Skip the first 5 values, which are box parameters
        class_id = np.argmax(scores)  # Get the index of the highest score
        confidence = scores[class_id]
        if confidence > 0.3:  # Lower confidence threshold to detect more objects
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))

In [25]:
# Step 8: Draw Bounding Boxes on the Image
for i in range(len(boxes)):
    x, y, w, h = boxes[i]
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(img, "Object detected", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

In [26]:
# Step 9: Display the Resulting Image
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
# Step 10: Dimension Estimation (Optional)
if len(boxes) > 0:
    # Use the dimensions of the first detected object (or the largest one if needed)
    first_box = boxes[0]  # Get the first detected object's bounding box
    w = first_box[2]  # width of the object
    h = first_box[3]  # height of the object

    ref_width = 10  # cm (the known width of a reference object)
    pixel_per_metric = w / ref_width  # Calculate pixels per centimeter
    real_width = w / pixel_per_metric
    real_height = h / pixel_per_metric

    print(f"Estimated Object Dimensions: Width = {real_width:.2f} cm, Height = {real_height:.2f} cm")
else:
    print("No objects detected to estimate dimensions.")

Estimated Object Dimensions: Width = 10.00 cm, Height = 12.82 cm


In [28]:
# Step 11: Suggest Box Size
box_sizes = [
    {"length": 15, "width": 10, "height": 5},
    {"length": 20, "width": 15, "height": 10},
    {"length": 30, "width": 20, "height": 15},
    # Add more box sizes as needed
]

def suggest_box_size(length, width, height):
    for box in box_sizes:
        if length <= box["length"] and width <= box["width"] and height <= box["height"]:
            return box
    return None

# Suggest the box size based on estimated dimensions
if len(boxes) > 0:
    box = suggest_box_size(real_width, real_height, 5)  # Assuming an object height of 5 cm
    if box:
        print(f"Suggested box size: {box}")
    else:
        print("No suitable box size found.")

Suggested box size: {'length': 20, 'width': 15, 'height': 10}
